In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import *

In [2]:
MISSING = -9999

In [3]:
df = pd.read_csv("../data/processed/data_encoded.csv")

In [4]:
# df = df.iloc[:13000]

In [5]:
# df = df.iloc[13000:]

In [6]:
df["satisfied"].value_counts()

1    15626
0    14454
Name: satisfied, dtype: int64

In [7]:
X = df.drop(columns=["id", "satisfied"]).values
y = df["satisfied"].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1001)

In [9]:
# model = xgb.XGBClassifier(learning_rate=0.1, 
#                           n_estimators=600, 
#                           objective='binary:logistic',
#                           silent=True, 
#                           missing=MISSING,
#                           nthread=1)

In [10]:
# model

In [11]:
# params = {
#     'max_depth': [2, 3, 4, 5]
# #     'subsample': [0.25, 0.5, 1.],
# #     'colsample_bytree': [0.5, 1.],
# #     'colsample_bylevel': [0.5, 1.],
# #     'colsample_bynode': [0.5, 1.],
# #     'scale_pos_weight': [0.5, 1., 1.5],
# }

In [12]:
# folds = 5
# param_comb = 5

# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# random_search = RandomizedSearchCV(model, 
#                                    param_distributions=params, 
#                                    n_iter=param_comb, 
#                                    scoring='roc_auc', 
#                                    n_jobs=5, 
#                                    cv=skf.split(X_train,y_train), 
#                                    verbose=3, 
#                                    random_state=1001 )

In [13]:
# random_search.fit(X_train, y_train)

In [14]:
# pd.DataFrame(random_search.cv_results_)

In [15]:
# final_model = random_search.best_estimator_

In [16]:
# y_pred = final_model.predict(X_test)
# y_prob = final_model.predict_proba(X_test)

In [45]:
params = {"max_depth": 6}

In [46]:
final_model = xgb.XGBClassifier(
    learning_rate=0.1,
    objective='binary:logistic',
    silent=True, 
    missing=MISSING,
    nthread=8,
    **params
    )

In [47]:
final_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=-9999, n_estimators=100, n_jobs=1,
              nthread=8, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=True, subsample=1, verbosity=1)

In [48]:
y_pred_train = final_model.predict(X_train)
y_prob_train = final_model.predict_proba(X_train)[:,1]

In [49]:
y_pred = final_model.predict(X_test)
y_prob = final_model.predict_proba(X_test)[:,1]

In [50]:
roc_auc_score(y_test, y_prob), roc_auc_score(y_train, y_prob_train)

(0.8879432548687523, 0.950724064047026)

In [23]:
0.8833639755978565 # baseline (fill missing)

0.8833639755978565

In [24]:
0.8839356100336466 # categorical variable

0.8839356100336466

In [25]:
0.8839356100336466 # type of missing using one-hot + indicator of missing

0.8839356100336466

In [26]:
pd.Series(final_model.feature_importances_, index=df.drop(columns=["id", "satisfied"]).columns).sort_values(ascending=False)

v98         0.267800
v79         0.100945
v101        0.046722
v224        0.040056
v223        0.028993
              ...   
v154_MON    0.000000
v154_NAP    0.000000
v154_NEP    0.000000
v154_NOR    0.000000
v1          0.000000
Length: 1507, dtype: float32